# Creating experiments using the runtool
This notebook is meant to help getting started with using the runtool.

## Defining a configuration file
We start with defining a configuration file which we want to use. This example assumes that you have a local docker image containing gluon-ts. The notebook has been tests on gluon-ts release v0.6.4 but should work for previous releases as well.

In [1]:
# Define the image and dataset files to use
# Replace these with your own values
image='gluonts_cpu:v0.6.4'
train_dataset='file:///Users/freccero/.mxnet/gluon-ts/datasets/electricity/train/data.json'
test_dataset='file:///Users/freccero/.mxnet/gluon-ts/datasets/electricity/test/data.json'

In [2]:
# generate a config. This should be done in a file, but for the purpose of this notebook
# Defining it as a string will suffice
config_yaml = f'''
simple:
    image: {image}
    instance: local
    hyperparameters:
        forecaster_name: gluonts.model.simple_feedforward.SimpleFeedForwardEstimator    
        freq: 
            $eval: $trial.dataset.meta.freq
        prediction_length:
            $eval: 2 * $trial.dataset.meta.prediction_length

deepar:
    image: {image}
    instance: local
    hyperparameters:
        forecaster_name: gluonts.model.deepar.DeepAREstimator  
        freq: 
            $eval: $trial.dataset.meta.freq
        prediction_length:
            $eval: 2 * $trial.dataset.meta.prediction_length

electricity_dataset:
    meta:
        freq: H
        prediction_length: 24
    path:
        train: {train_dataset}
        test: {test_dataset}
'''

The runtool can load config files either from a file using a path which is the prefered way. But it can also load a config if it is provided as a dictionary. Thus we have to convert the yaml above into a dict.

In [3]:
import yaml
config_data = yaml.safe_load(config_yaml)

## Writing the run script
Now when we have a configuration defined, we can implement a script which uses the runtool to define and dispatch experiments. 

In [4]:
import runtool
import boto3

In order to use the configurations stored in the `config` which we want to use we need to load the data using the runtool.

In [5]:
config = runtool.load_config(config_data)

Now we want to use the `config` to define experiments to run. This is done using the `*` and the `+` symbols. 

`+` concatenates either a set of algorithms or a set of datasets together.

`*` takes a set of algorithms and a set of datasets and generates an experiment from them. 

In this example we want the `deepar` and the `simple` algorithm to train on the `electricity_dataset`.

In [6]:
experiment = config.electricity_dataset * (config.simple + config.deepar)

Next we need to provided the runtool with the session it should use when running as well as providing a sagemaker role with proper permissions. Further, we need to provide a bucket where sagemaker will store the output data of the training jobs. If running jobs locally, the bucket and the role_arn can be left as empty strings.

In [7]:
tool = runtool.Client(
    role_arn="",
    bucket="",
    session=boto3.Session(),
)

It may at this point be beneficial to inspect what jobs will be created.
Performing a `dry-run` displays a summary of the jobs that are to be generated in a table.

In [15]:
# print-table does not work well in notebooks, thus we only look at the returned dataframe
df = tool.dry_run(experiment, print_table=False)
df.head()

,image,hyperparameters,output_path,instance,job_name,tags,run,datasets
0,gluonts_cpu:v0.6.4,{'forecaster_name': 'gluonts.model.simple_feed...,s3:///default_name/default_name_82171f73,local,config-b493ff82-date-2021-01-12-11-34-54-runid...,{'run_configuration_id': 'default_name_82171f7...,0,[file:///Users/freccero/.mxnet/gluon-ts/datase...
1,gluonts_cpu:v0.6.4,{'forecaster_name': 'gluonts.model.deepar.Deep...,s3:///default_name/default_name_e1b42233,local,config-cce8327f-date-2021-01-12-11-34-54-runid...,{'run_configuration_id': 'default_name_e1b4223...,0,[file:///Users/freccero/.mxnet/gluon-ts/datase...


When we are satisified with the jobs that are to be run, it is time to execute the jobs using the runtool.

In [ ]:
output_location = "/Users/freccero/local_sagemaker_output"

# for running locally
runs = tool.local_run(
    experiment, output_dir=output_location
)

# for running in sagemaker
#runs = tool.run(
#    experiment, output_dir=output_location
#)

running training jobs in local mode
Starting next training job (config-b493ff82-date-2021-01-12-11-34-54-runid-55c163da-run-0)

Creating tmpzdj_7do1_algo-1-610vu_1 ... 
Attaching to tmpzdj_7do1_algo-1-610vu_12mdone
algo-1-610vu_1  | [2021-01-12 11:44:40] [INFO] __main__ Run 'train' command
algo-1-610vu_1  | [2021-01-12 11:44:40] [INFO] gluonts.mx.context Using CPU
algo-1-610vu_1  | [2021-01-12 11:44:40] [INFO] gluonts.shell.train Using gluonts v0.6.5.dev0+g5fc89cc.d20201210
algo-1-610vu_1  | [2021-01-12 11:44:40] [INFO] gluonts.shell.train Using forecaster gluonts.model.simple_feedforward._estimator.SimpleFeedForwardEstimator v0.6.5.dev0+g5fc89cc.d20201210
algo-1-610vu_1  | [2021-01-12 11:44:40] [INFO] gluonts.shell.train Using the following data channels: train, test
algo-1-610vu_1  | [2021-01-12 11:44:40] [INFO] gluonts.shell.train The forecaster can be reconstructed with the following expression: gluonts.model.simple_feedforward._estimator.SimpleFeedForwardEstimator(batch_normalizat

In [ ]:
print(runs)